# Algorithme PyCrypto 

Dans cette partie nous allons implémenter notre algorithme PyCrypto en se basant sur la définition du BestMarkowitz.

In [1]:
import numpy as np
import pandas as pd

from universal import tools
from universal.algos import CRP


class Pycrypto(CRP):
    """
    Reference:
        https://en.wikipedia.org/wiki/Modern_portfolio_theory
    """

    PRICE_TYPE = "ratio"
    REPLACE_MISSING = False
    

    
    def __init__(self, global_sharpe=None, sharpe=None, **kwargs):
        self.global_sharpe = global_sharpe
        self.sharpe = sharpe
        self.opt_markowitz_kwargs = kwargs

    def weights(self, X, y):
        # y est la tables des prédictions faites par notre algorithme de machine learning
        # X est notre dataframe de close
        y = y.to_numpy()
        y = y.reshape(y.shape[0], 1)
        # update frequency
        freq = tools.freq(X.index)

        R = X - 1

        # calculate mean and covariance matrix and annualize them
        sigma = R.cov() * freq

        if self.sharpe:
            mu = pd.Series(np.sqrt(np.diag(sigma)), X.columns) * pd.Series(
                self.sharpe
            ).reindex(X.columns)
        elif self.global_sharpe:
            mu = pd.Series(np.sqrt(np.diag(sigma)) * self.global_sharpe, X.columns)
        else:
            mu = R.mean() * freq

        self.b = tools.opt_markowitz(mu, sigma, **self.opt_markowitz_kwargs)

        return super().weights(R)*y
